In [2]:
import dynamiqs as dq
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import numpy as np 

In [105]:
na = 20 # Hilbert space dimension for memory mode
nb = 5  # Hilbert space dimension for buffer mode

#Annihilation operators
a = dq.tensor(dq.destroy(na), dq.eye(nb))    
b = dq.tensor(dq.eye(na), dq.destroy(nb)) 

#Identity matching [a,b] dimensions
identity = dq.tensor(dq.eye(na), dq.eye(nb))

# Both modes start in a vacuum 
psi0 = dq.tensor(dq.fock(na, 0), dq.fock(nb, 0))

# Using A100, so double is supported
dq.set_precision('double') 

# Define constants
w_a0 = 5.26 
w_b0 = 7.70  
psi_a = 0.06
psi_b = 0.29
E_j = 42.76  
DE_j = 0.47  
w_d = 7.623  
w_p = 2.891 
e_d = -3.815e-3  # MHz to GHz
e_p = 0.122  # rad
kappa_a = 9.3e-6 # KHz to GHz
kappa_b = 2.6e-3 # MHz to GHz

tsave = jnp.linspace(0, 4, 100)

def ATS(t):
    e_t = e_p * jnp.cos(w_p * t)
    A = psi_a * (a + a.dag()) + psi_b * (b + b.dag())
    term1 = -2*E_j * jnp.sin(e_t) * jnp.sin(dq.to_jax(A)) 
    term2 = 2 * DE_j * jnp.cos(e_t) * jnp.cos(dq.to_jax(A))
    ATS = term1 + term2
    return ATS 

def d(t):
    d = 2 * e_d * jnp.cos(w_d * t)
    return d * (b + dq.dag(b))

def get_H(t):
    H_0 = w_a0 * (dq.dag(a) @ a) + w_b0 * (dq.dag(b) @ b)
    H_d = d(t)
    H_ATS = ATS(t)
    return H_0 + H_ATS + H_d

# H depends explicitly on t, so we need timecallable()
H = dq.timecallable(get_H)

#Typical 2-mode loss operator
loss_ops = [jnp.sqrt(kappa_a) * a, jnp.sqrt(kappa_b) * b]

res = dq.mesolve(H, loss_ops, psi0, tsave)



/local_scratch/slurm.1820260/ipykernel_1110836/4249743259.py:49: UserWarning: A sparse qarray has been converted to dense layout due to element-wise addition with a dense qarray.
  return H_0 + H_ATS + H_d
/home/najones/.local/lib/python3.11/site-packages/dynamiqs/qarrays/qarray.py:481: UserWarning: A sparse qarray has been converted to dense layout due to element-wise addition with a dense qarray.
  return self.__add__(y)
/local_scratch/slurm.1820260/ipykernel_1110836/4249743259.py:49: UserWarning: A sparse qarray has been converted to dense layout due to element-wise addition with a dense qarray.
  return H_0 + H_ATS + H_d
/home/najones/.local/lib/python3.11/site-packages/dynamiqs/qarrays/qarray.py:481: UserWarning: A sparse qarray has been converted to dense layout due to element-wise addition with a dense qarray.
  return self.__add__(y)
/local_scratch/slurm.1820260/ipykernel_1110836/4249743259.py:49: UserWarning: A sparse qarray has been converted to dense layout due to element-wi

In [107]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from IPython.display import HTML, display
# 6. Animate the Wigner function of mode a
# ---------------------------
fig_w, ax_w = plt.subplots(1, 1, figsize=(6, 6))

def update(frame):
    # Obtain the reduced state of mode a by tracing out mode b.
    ax_w.cla()  # Clear the axis.
    rho_a = dq.ptrace(res.states[frame], 0)
    dq.plot.wigner(rho_a, ax=ax_w)
    ax_w.set_title(f"Mode a Wigner Function\nTime = {tsave[frame]:.2f}")

ani = animation.FuncAnimation(fig_w, update, frames=len(tsave), repeat=False)
gif_filename = 'wigner_mode_a.gif'
ani.save(gif_filename, writer=PillowWriter(fps=len(tsave) / 4))
plt.close(fig_w)
display(HTML(f'<img src="{gif_filename}">'))